# CS 555: Homework 3
### Eric Stevens
### November 1, 2018

In [1]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals
from string import punctuation
import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict
from bitweight import BitWeight, BitWeightRangeError

In [2]:

small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs']


In [3]:
# TOKENIZE - converts lists of sentences, like in the small_corpus
# to list of list of tokens. All of the other functions will require
# their parameters in the form of the outupt of the tokenize function.
def tokenize(corpus):
    tokens = [sentence.split(' ') for sentence in corpus]
    return tokens



# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

### POS Filter for 'wsj.pos' and 'wsj-normalized.pos'

In [4]:
# FILE_TO_LIST - turns the wsj files into the form of the 'small_corpus'
# to prepare it to be the input parameter of the 'tokenize()' function.
def file_to_list(filename):
    with open(filename, 'r') as content_file:
        content = content_file.read()
        no_tags = re.sub('(<[A-Z$]{2,4}>)|(/[A-Z$]{2,4})|(\.\s+[a-z])|(/[.,$])|[\\,/\'`]', '', content)
        return re.split('[\n]',no_tags)
    
# NOTE: This function assumes that 'wsj.pos' and 'wsj-normalised.pos' have been unzipped.

### Demonstrate Filtering On 'wsj.pos'

In [5]:
# Top 10 values of 'wsj.pos'
wsj_filtered = file_to_list('wsj.pos')
wsj_filtered[:10]

[u'Digital Equipment Corp. reported a 32 % decline in net income on a modest revenue gain in its fiscal first quarter  causing some analysts to predict weaker results ahead than they had expected .',
 u'Although the second-largest computer maker had prepared Wall Street for a poor quarter  analysts said they were troubled by signs of flat U.S. orders and a slowdown in the rate of gain in foreign orders . The Maynard  Mass.  company is in a transition in which it is trying to reduce its reliance on mid-range machines and establish a presence in workstations and mainframes .',
 u'Net for the quarter ended Sept. 30 fell to $ 150.8 million  or $ 1.20 a share  from $ 223 million  or $ 1.71 a share  a year ago . Revenue rose 6.4 % to $ 3.13 billion from $ 2.94 billion .',
 u'Digital said a shift in its product mix toward low-end products and strong growth in workstation sales yielded lower gross margins . A spokesman also said margins for the company s service business narrowed somewhat beca

### Demonstrate Filtering On 'wsj-normalized.pos'

In [6]:
# Top 10 values of 'wsj-normalized.pos'
wsj_normalized_filtered = file_to_list('wsj-normalized.pos')
wsj_normalized_filtered[:10]

[u'   reported a  % decline in net income on a modest revenue gain in its fiscal first quarter  causing some analysts to predict weaker results ahead than they had expected .',
 u'Although the second-largest computer maker had prepared   for a poor quarter  analysts said they were troubled by signs of flat  orders and a slowdown in the rate of gain in foreign orders . The     company is in a transition in which it is trying to reduce its reliance on mid-range machines and establish a presence in workstations and mainframes .',
 u'Net for the quarter ended   fell to $    or $  a share  from $    or $  a share  a year ago . Revenue rose  % to $   from $   .',
 u'Digital said a shift in its product mix toward low-end products and strong growth in workstation sales yielded lower gross margins . A spokesman also said margins for the company s service business narrowed somewhat because of heavy investments made in that sector .',
 u'The lack of a strong product at the high end of  s line was

<font color="red">Self assessment:</font>

### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

### Simple Counting, Maximum Likelihood and Utility Functions

In [7]:
# TOKENIZE - converts lists of sentences, like in the small_corpus
# to list of list of tokens. All of the other functions will require
# their parameters in the form of the outupt of the tokenize function.
def tokenize(corpus):
    tokens = [sentence.split(' ') for sentence in corpus]
    return tokens


# COUNT_BUILDER - generates count models where 'corpus' is in 
# the form output by 'tokenize()'. Order is the 'n' in n-gram.
def count_builder(corpus, order):
    
    #ngram
    ng = ngrams(corpus, order)

    # describe model datatype
    model = defaultdict(lambda: defaultdict(int))
    
    # loop to build embedded defaultdict    
    for gram in ng: 
        if not gram[1] in model[gram[0]]:
            model[gram[0]][gram[1]] = 1
        else:
            model[gram[0]][gram[1]] += 1

    # Count Model
    return model


# MAX_LIKELIHOOD - converts a count model into its MLE form.
# 'count_model' is in the form output by 'count_builder()'
def max_likelihood(count_model):
    
    # Container for MLE model with BitWeight probabilities
    # Returns 0 for unseen values.
    model = defaultdict(lambda: defaultdict(lambda: BitWeight(0)))
    
    # for prefixes in model...
    for prefix, suffix_dict in count_model.iteritems():
        w_minus = BitWeight(0) # used to count total tokens
        
        # for words with hist prefix ...
        for suffix, count in suffix_dict.iteritems():
            w_minus += BitWeight(count) # add to total number of tokens
        
        # again, for words with hist prefix ...
        for suffix, count in suffix_dict.iteritems():
            model[prefix][suffix] = BitWeight.__itruediv__(BitWeight(count),w_minus) # set output probabilities
    
    # MLE Probabilities
    return model


# MODEL_PRINTER: Utility to print models with 'BitWeight' values
def model_printer(model):
    for prefix, suffix_dict in model.iteritems():
        for suffix, value in suffix_dict.iteritems():
            print(prefix, " : ", suffix, " : ", value.real())
        print(prefix, " : ", "xxxx", ' : ', model[prefix]['xxxx'].real())

### Demonstrate MLE Model Build on 'small_corpus'

In [8]:
# Create a trigram MLE language model for 'small_corpus' and show result
small_tokens = tokenize(small_corpus)
small_count = count_builder(small_tokens, 3)
small_lm = max_likelihood(small_count)
model_printer(small_lm)
print('\n\n')

for x, y in small_count.iteritems():
    for z,v in y.iteritems():
        print(x, ' : ', z, ' : ', v)
    print(x, " : ", "xxxx", ' : ', small_count[x]['xxxx'])

(u'us', u'start')  :  with  :  1.0
(u'us', u'start')  :  xxxx  :  0.0
(u'few', u'other')  :  examples  :  1.0
(u'few', u'other')  :  xxxx  :  0.0
(u'start', u'with')  :  a  :  0.333333333333
(u'start', u'with')  :  an  :  0.333333333333
(u'start', u'with')  :  in  :  0.333333333333
(u'start', u'with')  :  xxxx  :  0.0
('<S_1>', u'We')  :  never  :  1.0
('<S_1>', u'We')  :  xxxx  :  0.0
('<S_0>', '<S_1>')  :  Tokens  :  0.2
('<S_0>', '<S_1>')  :  We  :  0.2
('<S_0>', '<S_1>')  :  Let  :  0.2
('<S_0>', '<S_1>')  :  Why  :  0.4
('<S_0>', '<S_1>')  :  xxxx  :  0.0
(u'start', u'here')  :  </S_1>  :  1.0
(u'start', u'here')  :  xxxx  :  0.0
(u'Let', u'us')  :  start  :  1.0
(u'Let', u'us')  :  xxxx  :  0.0
(u'that', u'we')  :  start  :  1.0
(u'that', u'we')  :  xxxx  :  0.0
(u'example', u'with')  :  so  :  1.0
(u'example', u'with')  :  xxxx  :  0.0
(u'example', u'docs')  :  </S_1>  :  1.0
(u'example', u'docs')  :  xxxx  :  0.0
(u'in', u'example')  :  docs  :  1.0
(u'in', u'example')  :  xxxx

### Demonstrate MLE Model Build on 'wsj.pos'

In [9]:
# Create trigram MLE language model for 'wsj.pos' and show subset
wsj_tokens = tokenize(wsj_filtered)
wsj_count = count_builder(wsj_tokens, 3)
wsj_lm = max_likelihood(wsj_count)

# grab 10 keys
top_keys = wsj_lm.keys()[:10]

# subset of language model
sub_wsj = defaultdict(lambda: defaultdict(lambda: BitWeight(0)))
for key in top_keys: sub_wsj[key] = wsj_lm[key]
                      
# print subsample of wsj_model
model_printer(sub_wsj)


(u'have', u'made')  :    :  0.0217391304348
(u'have', u'made')  :  some  :  0.0434782608696
(u'have', u'made')  :  it  :  0.130434782609
(u'have', u'made')  :  trading  :  0.0217391304348
(u'have', u'made')  :  use  :  0.0434782608696
(u'have', u'made')  :  for  :  0.0217391304348
(u'have', u'made')  :  no  :  0.0652173913043
(u'have', u'made')  :  leveraged  :  0.0217391304348
(u'have', u'made')  :  .  :  0.0217391304348
(u'have', u'made')  :  their  :  0.0434782608696
(u'have', u'made')  :  much  :  0.0217391304348
(u'have', u'made')  :  China  :  0.0217391304348
(u'have', u'made')  :  health  :  0.0217391304348
(u'have', u'made')  :  available  :  0.0217391304348
(u'have', u'made')  :  them  :  0.0217391304348
(u'have', u'made')  :  his  :  0.0217391304348
(u'have', u'made')  :  big  :  0.0217391304348
(u'have', u'made')  :  metrics  :  0.0217391304348
(u'have', u'made')  :  nearly  :  0.0217391304348
(u'have', u'made')  :  excellent  :  0.0217391304348
(u'have', u'made')  :  such  

<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

### Whitten-Bell Ngram Model Builder Function

In [20]:

def wb_model_builder(tokens, order):
    
    # build count model and MLE model to calculate WB values
    count_mod = count_builder(tokens, order)
    mle_mod = max_likelihood(count_mod)
    
    
    # Base Case: Unigram Model Builder
    if order == 1:
        
        # get tokens and types to calc lmbda(.)
        tks = sum(count_mod[()].values())
        tps = len(count_mod[()].keys())
        
        # use above to build lambda(.)
        lambda_dot = BitWeight(tks,(tks+tps))
        one_minus_lambda_dot = BitWeight(tps,(tks+tps))
        one_over_V = BitWeight(1,tps)
        
        # create return for unseen values Pwb(.) = (1 - lambda) * (1/V)
        default_return = one_minus_lambda_dot*one_over_V

        # declare WB unigram model to be handed back
        #print(default_return.real())
        wb_model = defaultdict(lambda: defaultdict(lambda: default_return))
        
        # build WB unigram model
        for prefix, suffix_dict in mle_mod.iteritems() :
            for suffix, mle_prob in suffix_dict.iteritems():
                
                # Set WB Probability: Pwb(w) = lambda(.)*Pmle + one_minus_lambda(.)*one_over_V
                wb_model[prefix][suffix] = (lambda_dot * mle_prob) + (one_minus_lambda_dot*one_over_V)
        
        # return WB unigram model
        return wb_model
    
    
    # Recursive Case: If the order is greater than 1
    else:

        # get the model of P(w|h_n-1)
        wb_minus_one_model = wb_model_builder(tokens, order-1)

        # retrieve default value from lower order model
        default_return = wb_minus_one_model["x"]["x"] # xx will not be in any model

        # declare WB model with inhereted default
        wb_model = defaultdict(lambda: defaultdict(lambda: default_return))
        #print(default_return.real())
        
        # for each history... 
        for prefix, suffix_dict in count_mod.iteritems():

            # calculate history parameters
            c_hist = sum(suffix_dict.values())
            N_one_plus = len(suffix_dict.keys())

            # build lambda terms
            lambda_hist = BitWeight(c_hist,(c_hist+N_one_plus))
            one_minus_lambda_hist = BitWeight(N_one_plus,(c_hist+N_one_plus))

            # default
            default_return = one_minus_lambda_hist*wb_minus_one_model[prefix[1:]]['xxxxx']
            
            # set default returns for inner dictionary
            wb_model[prefix] = defaultdict(lambda: default_return)

            
            # for each word that follows that history ... 
            for suffix, wb_minus_prob in mle_mod[prefix].iteritems():

                # Set WB Probability: Pwb(w|hist) = lambda_hist*Pmle(w|hist) + one_minus_lambda_hist*Pwb(w|hist-1)
                lambda_hist_x_mle = lambda_hist*mle_mod[prefix][suffix]
                one_minus_lambda_x_wb_minus = one_minus_lambda_hist*wb_minus_one_model[prefix[1:]][suffix]
                wb_model[prefix][suffix] = lambda_hist_x_mle + one_minus_lambda_x_wb_minus
                wb_model.default_factory= one_minus_lambda_hist*wb_minus_one_model[prefix][suffix]
    # return WB ngram model
    return wb_model


### Demonstrate WB Model Build on 'small_corpus'

In [21]:
# Create a trigram Witten-Bell language model for 'small_corpus' and show result
small_wb_lm = wb_model_builder(small_tokens, 4)
model_printer(small_wb_lm)

TypeError: 'bitweight.BitWeight' object is not callable

In [12]:
print(small_wb_lm[(u'we',)]['xxxx'].real())
print(small_wb_lm[('<S_0>',)]['xxxx'].real())
print(small_wb_lm[('that')]['xxxx'].real())

0.00189393939394
0.00189393939394
0.00189393939394


### Demonstrate WB Model Build on 'wsj.pos'

In [13]:
# Create trigram Witten-Bell language model for 'wsj.pos' and show subset
wsj_wb_lm = wb_model_builder(wsj_tokens, 2)

# grab 10 keys
top_keys = wsj_wb_lm.keys()[:10]

# subset of language model
sub_wsj_wb = defaultdict(lambda: defaultdict(lambda: BitWeight(0)))
for key in top_keys: sub_wsj_wb[key] = wsj_wb_lm[key]
                      
# print subsample of wsj_wb_model
model_printer(sub_wsj_wb)

8.05431832277e-07
(u'computer-guided',)  :  funds  :  0.0589505033712
(u'computer-guided',)  :  program  :  0.294277075183
(u'computer-guided',)  :  selling  :  0.117737551459
(u'computer-guided',)  :  trading  :  0.117976101416
(u'computer-guided',)  :  buying  :  0.117729734032
(u'computer-guided',)  :  xxxx  :  2.21098934351e-07
(u'wolves',)  :  to  :  0.511219665424
(u'wolves',)  :  xxxx  :  2.21098934351e-07
(u'midweek',)  :  break  :  0.250029800978
(u'midweek',)  :  and  :  0.258014852163
(u'midweek',)  :  xxxx  :  2.21098934351e-07
(u'scoring',)  :  a  :  0.509620077805
(u'scoring',)  :  xxxx  :  2.21098934351e-07
(u'Hartley',)  :  in  :  0.257773625329
(u'Hartley',)  :  .  :  0.269467690102
(u'Hartley',)  :  xxxx  :  2.21098934351e-07
(u'iota',)  :  of  :  0.261440353746
(u'iota',)  :  .  :  0.269467690102
(u'iota',)  :  xxxx  :  2.21098934351e-07
(u'riffing',)  :  on  :  0.502630137648
(u'riffing',)  :  xxxx  :  2.21098934351e-07
(u'21.625',)  :  .  :  0.519467690102
(u'21.62

<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

### Functions for N Fold Cross Validation and Perplexity Calculation

In [14]:

#N_FOLD: generalized n_fold cross validation
#
#   # INPUTS
#   data: the dataset as a list
#   n: number of folds
#   scoring_function: function used to evaluate
#   param_list: list of parameters for scoring_function
#
#   # OUTPUTS
#   score_list: a list of the scores for the different test runs
def n_fold(data, n, scoring_function, param_list):
    
    # get lenght of corpus and use it to get test set size
    size_of_data = len(data)
    size_of_test_set = int(size_of_data/n)
    
    # score list holds onto results
    score_list = []
    
    ###  n folds  ##
    for x in range(n):
        
        # build test set and trainging set
        test_set = data[x*size_of_test_set:(x+1)*size_of_test_set]
        training_set = data[:x*size_of_test_set]+data[(x+1)*size_of_test_set:]        
        
        # evaluate
        score_list.append(scoring_function(test_set, training_set, *param_list))
          
    return score_list

def n_fold_on_training(data, n, scoring_function, param_list):
    
    # get lenght of corpus and use it to get test set size
    size_of_data = len(data)
    size_of_test_set = int(size_of_data/n)
    
    # score list holds onto results
    score_list = []
    
    ###  n folds  ##
    for x in range(n):
        
        # build test set and trainging set
        test_set = data[(x+1)*size_of_test_set:(x+2)*size_of_test_set]
        training_set = data[:x*size_of_test_set]+data[(x+1)*size_of_test_set:]        
        
        # evaluate
        score_list.append(scoring_function(test_set, training_set, *param_list))
          
    return score_list

# perplexity evaluation function
def wb_perplexity(test_set, training_set, order):
    
    # count number of tests
    N = 0.0
    
    # build witten_bell model from training data
    training_tokens = tokenize(training_set)
    wb_model = wb_model_builder(training_tokens, order)
    
    # get ngram of test set
    test_tokens = tokenize(test_set)
    test_grams = ngrams(test_tokens, order)

    # declare probability product holder
    p_w_all = BitWeight(1)
    
    # loop through all test data
    for history, word in test_grams:
        N += 1.0 # increment test count
        p_w_all *= wb_model[history][word]
    
    # perplexity in log from
    return 2**((1/N)*p_w_all.log())


### Whitten-Bell Perplexity 10 Fold Cross validation for 'wsj.pos'
##### Unigram, Bigram, Trigram, and 4-gram

In [15]:
# 10 Fold Cross Validation for 'wsj.pos' showing results
# Whitten-Bell Unigram
tl = file_to_list('wsj.pos')
n_fold(tl,10, wb_perplexity, [1])

[1372.0304278060817,
 1408.4353491077204,
 1342.0438956479438,
 1352.5326341889465,
 1353.042385306244,
 1335.405358118573,
 1379.534763788414,
 1288.0579450006292,
 1335.3570317131941,
 1407.2529563536805]

In [28]:
n_fold_on_training(tl,10, wb_perplexity, [1])

[1317.2301912839666,
 1264.9076919066579,
 1270.0635854247453,
 1272.1022537051106,
 1252.9800157939496,
 1290.9607213455297,
 1208.9090968261557,
 1255.4108930940413,
 1318.803874424966,
 15.41913098445743]

In [59]:
# Whitten-Bell Bigram
n_fold(tl,10, wb_perplexity, [2])

8.93623904082e-07
8.99867269578e-07
8.92938464039e-07
8.95723457922e-07
8.90546492761e-07
8.89089625569e-07
8.95998738434e-07
8.90832090623e-07
8.91057524892e-07
8.91201259089e-07


[1431.2469228737832,
 1581.230780204272,
 1224.72678411563,
 1263.5879726811686,
 1238.677669045329,
 1146.554185552507,
 1398.103944521792,
 1148.5305245801228,
 1194.1254360989317,
 1319.7431680185557]

In [30]:
n_fold_on_training(tl,10, wb_perplexity, [2])

8.93623904082e-07
8.99867269578e-07
8.92938464039e-07
8.95723457922e-07
8.90546492761e-07
8.89089625569e-07
8.95998738434e-07
8.90832090623e-07
8.91057524892e-07
8.91201259089e-07


[82.09983907072032,
 79.91637338290319,
 79.93257265828282,
 79.01728928772545,
 78.4065941438138,
 81.01781542663208,
 78.38058769162845,
 79.75806155798375,
 79.55906699433311,
 25.926204046870843]

In [31]:
# Whitten-Bell Trigram
n_fold(tl,10, wb_perplexity, [3])

8.93623904082e-07
2.51893986386e-07
8.99867269578e-07
2.48647535015e-07
8.92938464039e-07
2.40705151176e-07
8.95723457922e-07
2.68011743315e-07
8.90546492761e-07
2.51528398719e-07
8.89089625569e-07
2.45266103605e-07
8.95998738434e-07
2.63160468631e-07
8.90832090623e-07
2.38846285167e-07
8.91057524892e-07
2.45190325641e-07
8.91201259089e-07
2.63594738604e-07


[57603.73776770055,
 62625.21225646621,
 46515.73597052397,
 45063.8584990199,
 43910.34366291097,
 41465.7121272675,
 52087.81146738057,
 41942.10579946517,
 44039.89514166364,
 46940.52906474769]

In [32]:
n_fold_on_training(tl,10, wb_perplexity, [3])

8.93623904082e-07
2.51893986386e-07
8.99867269578e-07
2.48647535015e-07
8.92938464039e-07
2.40705151176e-07
8.95723457922e-07
2.68011743315e-07
8.90546492761e-07
2.51528398719e-07
8.89089625569e-07
2.45266103605e-07
8.95998738434e-07
2.63160468631e-07
8.90832090623e-07
2.38846285167e-07
8.91057524892e-07
2.45190325641e-07
8.91201259089e-07
2.63594738604e-07


[8.689907841206061,
 8.888395273121684,
 8.77903801649161,
 8.846000035263602,
 8.939281299390643,
 8.745220384528949,
 8.981870837100123,
 8.915332484072534,
 8.712891765909628,
 30.62733371250019]

In [57]:
# Whitten-Bell 4-gram
n_fold(tl,10, wb_perplexity, [4])

8.93623904082e-07
2.51893986386e-07
1.25946993193e-07
8.99867269578e-07
2.48647535015e-07
1.24323767507e-07
8.92938464039e-07
2.40705151176e-07
1.20352575588e-07
8.95723457922e-07
2.68011743315e-07
1.34005871658e-07
8.90546492761e-07
2.51528398719e-07
1.25764199359e-07
8.89089625569e-07
2.45266103605e-07
1.22633051803e-07
8.95998738434e-07
2.63160468631e-07
1.31580234315e-07
8.90832090623e-07
2.38846285167e-07
1.19423142583e-07
8.91057524892e-07
2.45190325641e-07
1.22595162821e-07
8.91201259089e-07
2.63594738604e-07
1.31797369302e-07


[1010418.2421349279,
 1052074.1627080762,
 870369.3316684695,
 814349.7627573514,
 808416.1294523223,
 805442.951149279,
 931118.3251847305,
 825250.5624531792,
 856838.1395028569,
 822236.0515255453]

In [58]:
n_fold_on_training(tl,10, wb_perplexity, [4])

8.93623904082e-07
2.51893986386e-07
1.25946993193e-07
8.99867269578e-07
2.48647535015e-07
1.24323767507e-07
8.92938464039e-07
2.40705151176e-07
1.20352575588e-07
8.95723457922e-07
2.68011743315e-07
1.34005871658e-07
8.90546492761e-07
2.51528398719e-07
1.25764199359e-07
8.89089625569e-07
2.45266103605e-07
1.22633051803e-07
8.95998738434e-07
2.63160468631e-07
1.31580234315e-07
8.90832090623e-07
2.38846285167e-07
1.19423142583e-07
8.91057524892e-07
2.45190325641e-07
1.22595162821e-07
8.91201259089e-07
2.63594738604e-07
1.31797369302e-07


[2.598472705340581,
 2.6615251577289674,
 2.664754017021337,
 2.6777766119764683,
 2.7034644331478863,
 2.64229444735329,
 2.7246485451869664,
 2.7060199118304875,
 2.6600676282907725,
 15.482568531387503]

### Whitten-Bell Perplexity 10 Fold Cross validation for 'wsj-normalize.pos'
##### Unigram, Bigram, Trigram, and 4-gram

In [67]:
# 10 Fold Cross Validation for 'wsj-normalized.pos' showing results
# Whitten-Bell Unigram
tln = file_to_list('wsj-normalized.pos')
n_fold(tln,10, wb_perplexity, [1])

[-8.838239615179509,
 -8.779879013333002,
 -8.677781166013778,
 -8.690320438886525,
 -8.676073450797272,
 -8.610384891066394,
 -8.660571886938842,
 -8.588568183357246,
 -8.661522843912271,
 -8.660891811585456]

In [69]:
# Whitten-Bell Bigram
n_fold(tln,10, wb_perplexity, [2])

[-8.764935670954612,
 -8.739983189526205,
 -8.42617158295725,
 -8.52500572331747,
 -8.456541351925848,
 -8.279701892776583,
 -8.476945416450668,
 -8.275239726492444,
 -8.374343776620037,
 -8.45390821701262]

In [72]:
# Whitten-Bell Trigram
n_fold(tln,10, wb_perplexity, [3])

[-12.533864751773706,
 -12.420369897097595,
 -12.202055718426166,
 -12.30242217205399,
 -12.138332015389349,
 -11.981262203140178,
 -12.207940135436033,
 -12.026194206318184,
 -12.085789090841663,
 -12.075071338557699]

In [167]:
# Whitten-Bell 4-gram
n_fold(tln,10, wb_perplexity, [4])

[-15.31460434529303,
 -15.215397812061918,
 -15.119107808367305,
 -15.031293781066399,
 -14.994357118905963,
 -14.851657548784262,
 -15.014035985130482,
 -14.95445233656557,
 -14.957400540425956,
 -14.833688024612025]

### Questions

##### 1. How does perplexity change as the model order size increases?
It can be seen from the results of both the 'wsj.pos' and the 'wsj-normalized.pos' that as the order of the model increases the perplexity decreases. This indicates that the higher order models we see here are more accurate than the lower ones.

##### 2. How does perplexity change as the data changed?
It can be seen in the results that the model was less acurate in prediction the outcomes of the normalized file. Im guessing that this is the result of trying to make the number of individual words more even, thus reducing the abbility that the Whitten Bell learning algorithm has of biasing words that it has seen more often.

<font color="red">Self assessment:</font>

In [15]:
z = len(tl)
sz=z/20
wb_perplexity(tl[:-sz], tl[:sz], 2)

1.46875229493e-05
1.46875229493e-05


-11.582747599507186

In [ ]:
nfold